In [1]:
import wikipediaapi
from bs4 import BeautifulSoup
import requests

In [2]:
languages = ['eo', 'ia', 'lfn', 'en', 'de', 'fr']
page_titles = ['United_Kingdom', 'Japan', 'South_Korea', 'Brazil', 'North_Korea', 'Hong_Kong']

# Use with specific urls, otherwise hardcode page_titles
# list_url = "https://en.wikipedia.org/wiki/Wikipedia:Wikipedia_articles_written_in_the_greatest_number_of_languages"
# site = requests.get(list_url)
# soup = BeautifulSoup(site.content, "html.parser")
# for p in soup.find_all('p'):
#     for a in p.find_all('a'):
#         page_titles.append(a.get('title'))

In [ ]:
def scrape_wikipedia_pages(languages, page_titles):
    corpus = {}
    for title in page_titles:
        exists_in_all_languages = True
        for lang in languages:
            wiki_lang = wikipediaapi.Wikipedia(language=lang)
            page = wiki_lang.page(title)
            if not page.exists():
                exists_in_all_languages = False
                break

        if exists_in_all_languages:
            corpus[title] = {}
            for lang in languages:
                wiki_lang = wikipediaapi.Wikipedia(language=lang, extract_format=wikipediaapi.ExtractFormat.HTML)
                page = wiki_lang.page(title)
                p_soup = BeautifulSoup(page.text, "html.parser")
                # div = p_soup.find('div', attrs={'class': 'mw-body-content'})
                p_tag = p_soup.find_all('p')
                text = ' '.join(p.get_text() for p in p_tag if p.get_text().strip())
                corpus[title][lang] = text
                
    return corpus

def save_corpus_to_files(corpus):
    for title, data in corpus.items():
        for lang, content in data.items():
            filename = f'{title}_{lang}.txt'
            with open(f'./parallel/{filename}', 'w', encoding='utf-8') as file:
                file.write(content)


corpus = scrape_wikipedia_pages(languages, page_titles)

save_corpus_to_files(corpus)